## Woermann Brock 

Serves as a normalization script before the execution of the preload.

In [1]:
import pandas as pd
import re
import datetime
import os

from datetime import datetime
from sftp_downloader import SFTPDownloader

In [2]:
to_upload_folder_path = r'R:\RawData\Elite Star\\Woermann Brock\ToUpload'
done_folder_path = r'R:\RawData\Elite Star\\Woermann Brock\\Done'

In [3]:
# Returns the latest file in the specified path based on the date modified
def get_latest_file(path):
  files = os.listdir(path)
  latest_file = max(files, key=lambda x: os.path.getmtime(os.path.join(path, x)))
  return latest_file

In [4]:
def extract_date_from_filename(filename):
    match = re.search(r'(\d{2})(\d{2})(\d{4})', filename)
    if match:
        date_str = match.group(1) + match.group(2) + match.group(3)
        date_obj = datetime.strptime(date_str, '%d%m%Y')
        return date_obj
    else:
        return datetime.min # If the date is not found, set it to a default value

file_list = os.listdir(to_upload_folder_path)
file_list.sort(key=lambda x: extract_date_from_filename(x), reverse=True)

last_file_processed = get_latest_file(done_folder_path)

sorted_files = [{'File Name': filename, 'Date': extract_date_from_filename(filename), 'Last File Processed': last_file_processed} for filename in file_list]

df = pd.DataFrame(sorted_files)
print(df)

                File Name                 Date      Last File Processed
0  ESTexport-03072024.txt  2024-07-03 00:00:00  ESTexport-25062024 .txt
1  ESTexport-02072024.txt  2024-07-02 00:00:00  ESTexport-25062024 .txt
2  ESTexport-01072024.txt  2024-07-01 00:00:00  ESTexport-25062024 .txt
3  ESTexport-30062024.txt  2024-06-30 00:00:00  ESTexport-25062024 .txt
4  ESTexport-29062024.txt  2024-06-29 00:00:00  ESTexport-25062024 .txt
5  ESTexport-28062024.txt  2024-06-28 00:00:00  ESTexport-25062024 .txt
6  ESTexport-27062024.txt  2024-06-27 00:00:00  ESTexport-25062024 .txt
7  ESTexport-26062024.txt  2024-06-26 00:00:00  ESTexport-25062024 .txt
8                 Back up  0001-01-01 00:00:00  ESTexport-25062024 .txt


In [5]:
def normalize_filename(filename):
    base, ext = os.path.splitext(filename)
    normalized_filename = f"{base.strip()} {ext.strip()}"
    return normalized_filename

file_data = []
for filename in os.listdir(to_upload_folder_path):
    if os.path.isfile(os.path.join(to_upload_folder_path, filename)):
        normalized_filename = normalize_filename(filename)
        if normalized_filename != filename:  
            file_data.append([filename, normalized_filename])
            destination_path = os.path.join(to_upload_folder_path, normalized_filename)
            if os.path.exists(destination_path):
                os.remove(destination_path)  # Remove the existing file
            os.rename(os.path.join(to_upload_folder_path, filename), destination_path)

df = pd.DataFrame(file_data, columns=['Initial Filename', 'Final Filename'])
print(df)


         Initial Filename           Final Filename
0  ESTexport-01072024.txt  ESTexport-01072024 .txt
1  ESTexport-02072024.txt  ESTexport-02072024 .txt
2  ESTexport-03072024.txt  ESTexport-03072024 .txt
3  ESTexport-26062024.txt  ESTexport-26062024 .txt
4  ESTexport-27062024.txt  ESTexport-27062024 .txt
5  ESTexport-28062024.txt  ESTexport-28062024 .txt
6  ESTexport-29062024.txt  ESTexport-29062024 .txt
7  ESTexport-30062024.txt  ESTexport-30062024 .txt
